## 9.1 インストール
```bash
$ pip install tensorflow
$ pip install tensorflow-gpu
```

## 9.2 最初のグラフの作成とセッション内での実行
TensorFlow では以下のような流れで計算を実行する．
- 1. 計算グラフの構築
- 2. セッションを開き計算を行う  
  
計算グラフを構築するだけでは変数の初期化さえ行われていない．セッションを開きオペレーションを CPU や GPU に載せて変数値を保持し計算を行う．

In [2]:
import tensorflow as tf
# 計算グラフを作る
x = tf.Variable(3, name="x")
y = tf.Variable(4, name="y")
f = x * x * y + y + 2

In [3]:
# TensorFlow セッションを開き実際に計算する
sess = tf.Session()
sess.run(x.initializer)
sess.run(y.initializer)
result = sess.run(f)
print(result)

# セッションを閉じる
sess.close()

42


with ブロックでセッションを開いて実行すれば，最後にセッションを閉じる必要はない．  
また，sess.run(f) と f.eval() は同じで計算の実行を担っている．

In [4]:
# 別の方法で実行する
# 最後に close しなくて良い
with tf.Session() as sess:
    x.initializer.run()
    y.initializer.run()
    result = f.eval()
    
print(result)

42


tf.global_variables_initializer() を使えば変数を一括して初期化できる．  

In [6]:
# global_variables_initializer() のインスタンスを作成
init = tf.global_variables_initializer()
with tf.Session() as sess:
    init.run()  # すべての変数を初期化
    result = f.eval()
    
print(result)

42


tf.InteractiveSession() で自動的にデフォルトセッションを呼び出せるが，最後はマニュアルで close する必要がある．

In [7]:
# IntaractiveSession() を使えば with ブロックは不要
sess = tf.InteractiveSession()
init.run()
result = f.eval()
print(result)

# ただし close しないといけない
sess.close()

42


## 9.3 グラフの管理
作成したノードは自動的にデフォルトグラフに追加される．  
複数の計算グラフを管理する場合には，tf.Graph() を呼び出して新しいグラフを作り，with ブロックで一時的にデフォルトグラフに設定する． 

In [8]:
x1 = tf.Variable(1)
print(x1.graph is tf.get_default_graph())

# 複数のグラフを作る場合には、以下のように新しい Graph を作れば良い
graph = tf.Graph()
with graph.as_default():
    x2 = tf.Variable(2)
print(x2.graph is graph)
print(x2.graph is tf.get_default_graph())

True
True
False


tf.reset_default_graph() を呼び出せばデフォルトグラフをリセットできる．

## 9.4 ノードの値のライフサイクル
すべてのノードの値はグラフを実行するたびに破棄される．したがって，下記のようなコードでは w と x はそれぞれ2回ずつ計算されることになる．

In [9]:
# TensorFlow は自動的にノードの依存関係を判断してくれる
w = tf.constant(3)
x = w + 2
y = x + 5
z = y * 3

with tf.Session() as sess:
    # w と x は自動的に評価され、最後に y が計算される
    print(y.eval())
    # w と x の評価結果は再利用されず、再度計算される
    print(z.eval())

10
30


上のような計算方法では w と x を2回評価するため非効率．下記のように変更することで1回の実行で y と z が同時に評価される．

In [10]:
# w と x を2回評価するのは非効率なので、1回の実行で y と z が同時に評価されるようにする
with tf.Session() as sess:
    y_val, z_val = sess.run([y, z])
    print(y_val)
    print(z_val)

10
30
